In [ ]:
import argparse
import base64
import json
import cv2

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
import math
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import tensorflow as tf
#tf.python.control_flow_ops = tf

new_size_col = 200
new_size_row = 66

def cropImage(image):
    # Preprocessing image files
    shape = image.shape
    # note: numpy arrays are (row, col)!
    image = image[math.floor(shape[0]/3):shape[0]-20, 0:shape[1]]
#     image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)    
    return image

def resizeImage(image):
    # Preprocessing image files
    shape = image.shape
    image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)   
    return image

def preprocess(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # get shape and chop off 1/3 from the top
    image = cropImage(image)
    image = resizeImage(image)
    # note: numpy arrays are (row, col)!
    #image = image[shape[0]-70:shape[0]-25, 0:shape[1]]
    #image = cv2.resize(image, (200,66), interpolation=cv2.INTER_AREA)
    #image = cv2.resize(image, (64,64), interpolation=cv2.INTER_AREA)
    return image



sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None





class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error
        
        cal_throttle = self.Kp * self.error + self.Ki * self.integral
        throttle = min(cal_throttle, 0.5)
        
        return throttle


controller = SimplePIController(0.1, 0.002)
set_speed = 18
controller.set_desired(set_speed)



@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_pre = np.asarray(image)
    image_array = preprocess(image_pre)
    #image_array = np.asarray(image)
    transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = 1.0*float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = controller.update(float(speed))
#     if float(speed) > 18:
#         throttle = 0.1
#     else:
#         throttle = 0.3
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Remote Driving')
#     parser.add_argument('model', type=str,
#     help='Path to model definition json. Model weights should be on the same path.')
#     args = parser.parse_args()
#     with open(args.model, 'r') as jfile:
#         model = model_from_json(jfile.read())
    with open('model.json', 'r') as jfile:
        model = model_from_json(jfile.read())

    model.compile("adam", "mse")
    weights_file = 'model.h5'
#     weights_file = './tmp/result2.20-0.04.hdf5'   
    model.load_weights(weights_file)

    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Using TensorFlow backend.
(16104) wsgi starting up on http://0.0.0.0:4567
(16104) accepted ('127.0.0.1', 35927)


connect  fde48de57bed4eb8988b2e873111c750
-0.16754618287086487 0.5
-0.16754618287086487 0.5
-0.16754618287086487 0.5
-0.1389392912387848 0.5
-0.1389392912387848 0.5
-0.1389392912387848 0.5
-0.1995064616203308 0.5
-0.1995064616203308 0.5
-0.1995064616203308 0.5
-0.2299787998199463 0.5
-0.2299787998199463 0.5
-0.2299787998199463 0.5
-0.14718219637870789 0.5
-0.14718219637870789 0.5
-0.14718219637870789 0.5
-0.19232133030891418 0.5
-0.17581120133399963 0.5
-0.17581120133399963 0.5
-0.20163699984550476 0.5
-0.20163699984550476 0.5
-0.20163699984550476 0.5
-0.21783095598220825 0.5
-0.21783095598220825 0.5
-0.21783095598220825 0.5
-0.036380693316459656 0.5
-0.036380693316459656 0.5
-0.036380693316459656 0.5
-0.14232125878334045 0.5
-0.14232125878334045 0.5
-0.14232125878334045 0.5
-0.12258259207010269 0.5
-0.12258259207010269 0.5
-0.12258259207010269 0.5
-0.04020111262798309 0.5
-0.04020111262798309 0.5
-0.04020111262798309 0.5
-0.17843374609947205 0.5
-0.17843374609947205 0.5
-0.17843374609